# Everruns Agent API Example

This notebook demonstrates how to use the Everruns API to:
1. Create an agent with a system prompt
2. Create a session (conversation)
3. Send a message to trigger the agentic loop
4. Read events until the assistant response appears
5. Continue the conversation

## Prerequisites

- Everruns API server running at `http://localhost:9000`
- Python packages: `requests`

In [ ]:
# Install required packages
!pip install requests

In [ ]:
import requests
import json
import time

# Configuration
BASE_URL = "http://localhost:9000"
API_V1 = f"{BASE_URL}/v1"

def print_json(data):
    """Pretty print JSON data."""
    print(json.dumps(data, indent=2, default=str))

## 1. Health Check

Verify the API server is running.

In [ ]:
response = requests.get(f"{BASE_URL}/health")
response.raise_for_status()
print("API Status:", response.json())

## 2. Create an Agent

An agent is a configuration for an agentic loop with a system prompt.

In [ ]:
agent_data = {
    "name": "Python Example Agent",
    "description": "A helpful assistant created via the Python API",
    "system_prompt": "You are a helpful assistant. Be concise and friendly.",
    "tags": ["example", "python"]
}

response = requests.post(f"{API_V1}/agents", json=agent_data)
response.raise_for_status()
agent = response.json()

agent_id = agent["id"]
print(f"Created agent: {agent_id}")

## 3. Create a Session

A session is an instance of conversation with the agent.

In [ ]:
session_data = {
    "title": "Python API Test Session"
}

response = requests.post(f"{API_V1}/agents/{agent_id}/sessions", json=session_data)
response.raise_for_status()
session = response.json()

session_id = session["id"]
print(f"Created session: {session_id}")

## 4. Send a Message

Sending a user message triggers the agentic loop workflow.

In [ ]:
message_data = {
    "message": {
        "role": "user",
        "content": [
            {"type": "text", "text": "What is 2 + 2? Please explain briefly."}
        ]
    }
}

response = requests.post(
    f"{API_V1}/agents/{agent_id}/sessions/{session_id}/messages",
    json=message_data
)
response.raise_for_status()
print(f"Sent message: {response.json()['id']}")

## 5. Read Events

Poll the events endpoint to get the agent's response. Events include messages (`message.agent`, `message.tool_call`, `message.tool_result`) and loop lifecycle events.

In [ ]:
def read_until_output(agent_id: str, session_id: str, timeout: int = 60):
    """Poll events continuously until an agent message appears.
    
    Prints all events as they arrive. Returns the agent's response text.
    """
    url = f"{API_V1}/agents/{agent_id}/sessions/{session_id}/events"
    seen_sequences = set()
    start_time = time.time()
    
    while time.time() - start_time < timeout:
        response = requests.get(url)
        response.raise_for_status()
        events = response.json().get("data", [])
        
        for event in events:
            seq = event["sequence"]
            if seq in seen_sequences:
                continue
            seen_sequences.add(seq)
            
            event_type = event["event_type"]
            data = event["data"]
            
            print(f"[{seq}] {event_type}")
            
            # For message.agent, format nicely and return text
            if event_type == "message.agent":
                content = data.get("content", [])
                text_parts = [p["text"] for p in content if p.get("type") == "text"]
                text = "\n".join(text_parts)
                print(json.dumps(data, indent=2))
                print()
                return text
            else:
                # Print full event data for all other events
                print(json.dumps(data, indent=2))
                print()
            
            # Check for failure
            if event_type == "session.failed":
                return None
        
        time.sleep(0.5)
    
    return None

# Read events until we get the agent's response
print("Polling for events...\n")
agent_response = read_until_output(agent_id, session_id)

if agent_response:
    print("=" * 40)
    print("AGENT RESPONSE:")
    print(agent_response)
else:
    print("No response received")

## 6. Continue the Conversation

Send another message and read the response.

In [ ]:
# Send follow-up message
message_data = {
    "message": {
        "content": [
            {"type": "text", "text": "What about 3 + 3?"}
        ]
    }
}

response = requests.post(
    f"{API_V1}/agents/{agent_id}/sessions/{session_id}/messages",
    json=message_data
)
response.raise_for_status()
print("Sent follow-up message\n")

# Read response
agent_response = read_until_output(agent_id, session_id)

if agent_response:
    print(f"\n--- AGENT ---")
    print(agent_response)

## 7. Cleanup

Delete the session and archive the agent.

In [ ]:
# Delete session
requests.delete(f"{API_V1}/agents/{agent_id}/sessions/{session_id}")
print(f"Deleted session: {session_id}")

# Archive agent
requests.delete(f"{API_V1}/agents/{agent_id}")
print(f"Archived agent: {agent_id}")

## Summary

Core API workflow:

1. `POST /v1/agents` - Create agent
2. `POST /v1/agents/{id}/sessions` - Create session
3. `POST /v1/agents/{id}/sessions/{id}/messages` - Send user message (triggers agentic loop)
4. `GET /v1/agents/{id}/sessions/{id}/events` - Poll events to get agent response

### Event Types

Events follow the pattern `{entity}.{action}` and contain all session activity:

**Message Events:**
- `message.user` - User message
- `message.agent` - Agent response with content
- `message.tool_call` - Tool invocation by agent
- `message.tool_result` - Result of tool execution

**Session Events:**
- `session.started`, `session.completed`, `session.failed` - Session lifecycle

**Step Events:**
- `step.started`, `step.generating`, `step.generated`, `step.error` - Processing steps

API docs: http://localhost:9000/swagger-ui/